In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensor

import torch.nn.functional as F
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

In [2]:
# reproduce
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

In [3]:
df = pd.read_csv("../input/train.csv")
print(f"df shape is {df.shape}")
df.head()

df shape is (160, 2)


,img_name,target
0,RT3_1,1
1,RT3_2,1
2,RT3_3,1
3,RT3_4,1
4,RT3_5,1


In [4]:
IMAGE_SIZE = 224
class UmdDataset(Dataset):
    def __init__(self, df: pd.DataFrame, imfolder: str, transforms=None):
        self.df = df
        self.imfolder = imfolder
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        im_path = os.path.join(self.imfolder, self.df.iloc[index]['img_name'] + '.png')
        x = cv2.imread(im_path, 0)
        x = cv2.cvtColor(x, cv2.COLOR_GRAY2RGB)
        
        if self.transforms:
            augmented = self.transforms(image=x)
            x = augmented['image']
            
        y = self.df.loc[index]['target']
        return x, y
    
df_transform = A.Compose([
    A.RandomCrop(IMAGE_SIZE, IMAGE_SIZE, p=1),
    A.Normalize(),
    ToTensor(),
])

In [5]:
training_data_path = "../input/"
umd_dataset = UmdDataset(df=df, imfolder=training_data_path, transforms=df_transform)
train_loader = DataLoader(dataset=umd_dataset, batch_size=1, num_workers=16, shuffle=True)

In [6]:
# # visualize images
# for images, labels in train_loader:  
#     img = images.squeeze(0).permute((1, 2,0))
#     plt.imshow(img)
#     plt.show()

In [7]:
# # alexnet
# umd_features = []
# umd_y = []

# model = models.alexnet(pretrained=True)

# for epoch in tqdm(range(10)):
    
#     for images, labels in train_loader:  

# #         features = model.extract_features(images)
# #         features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).squeeze(1).squeeze(1).cpu().detach().numpy()
#         features = model.features(images)
#         features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).cpu().detach().numpy()
#         features = features.reshape([-1])
#         umd_features.append(features)
# #         umd_features.append(features.cpu().detach().numpy())
#         umd_y.append(labels.detach().numpy())

# umd_features = np.array(umd_features)
# umd_y = np. array(umd_y)

In [8]:
umd_features = []
umd_y = []

model = models.resnet18(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()

for epoch in tqdm(range(10)):
    
    for images, labels in train_loader:  

#         features = model.extract_features(images)
#         features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).squeeze(1).squeeze(1).cpu().detach().numpy()
        features = feature_extractor(images)
        features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).squeeze(1).squeeze(1).cpu().detach().numpy()
#         features = features.reshape([-1])
        umd_features.append(features)
#         umd_features.append(features.cpu().detach().numpy())
        umd_y.append(labels.detach().numpy())

umd_features = np.array(umd_features)
umd_y = np. array(umd_y)

100%|██████████| 10/10 [00:33<00:00,  3.36s/it]


In [9]:
print(f"image feature shape is {umd_features.shape}")
print(f"image label shape is {umd_y.shape}")

image feature shape is (1600, 512)
image label shape is (1600, 1)


In [10]:
df_umd = pd.DataFrame(umd_features)
df_umd["target"] = umd_y

print(df_umd.shape)
df_umd.head()

(1600, 513)


,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,target
0,1.852463,1.427170,3.690488,0.438352,2.316507,0.000000,0.240232,0.074002,4.813165,0.286297,...,1.502527,0.011416,1.727503,0.042662,2.811683,0.212586,0.903000,0.004605,0.000000,2
1,0.040199,0.191156,0.083933,0.492226,0.110319,0.112916,0.614150,0.309155,2.855123,0.000000,...,0.108537,0.000000,3.950866,2.096619,0.126770,0.015501,1.008542,0.363737,0.020453,3
2,0.307319,0.881700,1.862720,0.012181,0.270546,0.001163,0.012460,0.359653,4.932673,0.468637,...,0.178925,0.060749,1.050058,1.036049,0.031714,0.426136,1.348130,0.000363,0.205434,3
3,0.323085,0.101360,0.306357,0.756577,0.000000,0.021451,0.000000,0.545721,3.178084,0.366844,...,0.182106,0.002339,0.504327,0.499391,1.048448,0.031410,0.851934,0.118453,0.000000,3
4,0.184685,0.411287,4.593755,0.988969,3.391530,0.001681,0.354382,0.084353,5.852554,1.916361,...,0.005103,0.128949,1.671188,0.469654,0.493812,0.844183,0.945601,0.000000,0.095774,2


In [11]:
df_umd_1 = df_umd[df_umd.target == 1]
df_umd_2 = df_umd[df_umd.target == 2]
df_umd_3 = df_umd[df_umd.target == 3]

In [12]:
train = pd.concat([df_umd_1[:200], df_umd_2[:200]], axis=0)
test = pd.concat([df_umd_1[200:], df_umd_2[200:], df_umd_3[:200]], axis=0)

train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

print(f"train shape is {train.shape}, test shape is {test.shape}")

train shape is (400, 513), test shape is (600, 513)


In [13]:
df_umd_12 = df_umd[df_umd.target == 12].reset_index(drop=True) # merged class 1&2 image

# train.to_csv("../features/train_features.csv", index=False)
# test.to_csv("../features/test_features.csv", index=False)
# df_umd_12.to_csv("../features/blend38_features.csv", index=False)